<a href="https://colab.research.google.com/github/Bryan-Az/Ai-Agents/blob/main/Ai_Agents_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a simple custom Ai Agent pipeline.
In this notebook, I will create a simple agent that utilizes predefined tools to answer a question. The actual architecture is a pre-trained LLM to process the text query input. The LLM is given a prompt and pre-defined tool/functions to use when answering the question.

For this assignment, I will have the LLM (ChatGPT - token needed) answer questions using a function that queries information from an online source.

## Imports

In [1]:
! pip install openai==1.14.2
import openai
import re
import httpx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [13]:
openai.__version__

'1.14.2'

## Chain-Of-Thought Prompting

When creating my prompt, I decided to prepend the examples of how to utilize the tools with a general thought process using the Chain-of-Thought theory:

"You answer questions based on a chain of "Thought, Action, Pause, Observation",
followed by a second "Thought, Pause, Observation" chain of thought.
At the end of the second chain of thought you output an 'Answer'."
Use Thought to describe your thought process about the question you were asked.
Use Action to use one of the 'Action' tools available to you - then return Pause.
Observation will be a summary of the Thought & Action."

I believe this is an important addition to add as it helps structure the answer returned by the large language model. This method of structuring a prompt is also known as a CoT (Chain of Thought), which helps improve the performance by allowing the model to re-iterate its' answer
 based on its previous 'thoughts'.

 Since the model is asked to only call the 'Action' step of its' chain once, it may only observe a single set of data and craft a single response 'Answer'. This is also known as the chain-of-thought.

 Tree-of-Thought is a more advanced version that will have the model make multiple actions and re-contextualize it's answer based on new data.

In [2]:
prompt = """
You answer questions based on a chain of "Thought, Action, Pause, Observation",
followed by a second "Thought, Pause, Observation" chain of thought.
At the end of the second chain of thought you output an 'Answer'."
Use Thought to describe your thought process about the question you were asked.
Use Action to use one of the 'Action' tools available to you - then return Pause.
Observation will be a summary of the Thought & Action.
Your available actions are:
wikipedia:
e.g. Wikipedia: Nasa Projects
Returns a summary from searching Wikipedia.
calculate:
e.g. calculate: 9 + 12
Runs a calculation and returns the result.
Example Session:
Question: How does nasa analyse images?
Chain of Thought 1:
Thought: I should look up nasa image analysis on Wikipedia.
Action: wikipedia: Image Analysis is a technique in Data Science.
Pause
Observation: I found that image analysis is a technique in Data Science.
Chain of Thought 2:
Thought 2: I should look up image analysis and nasa on Wikipedia.
Pause
Observation 2: NASA has acquired samples from Mars, they analyzed multiple images and created a mosaic.
Answer: NASA has acquired samples from Mars. They seem to have used a technique where mosaic images are scanned, which is a technique also used in Data analysis for processing large amounts of data in parallel.
""".strip()


## The Chat Bot Agent

In [3]:
action_re = re.compile('^Action: (\w+): (.*)$')
def query(question, max_turns=1):
    i = 0
    bot = ChatBot(prompt)
    next_prompt = question
    result = bot(next_prompt)
    print(result)
    # This section can be reimplemented for Tree-Of-Thought
    #while i < max_turns:
    #    i += 1
    #    result = bot(next_prompt)
    #    print(result)
    #    actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
    #    if actions:
    #        # There is an action to run
    #        action, action_input = actions[0].groups()
    #        if action not in known_actions:
    #            raise Exception("Unknown action: {}: {}".format(action, action_input))
    #        print(" -- running {} {}".format(action, action_input))
    #        observation = known_actions[action](action_input)
    #        print("Observation:", observation)
    #        next_prompt = "Observation: {}".format(observation)
    #    else:
    #        return
    return

In [8]:
openai.api_key = "sk-..."

class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = openai.chat.completions.create(model="gpt-3.5-turbo", messages=self.messages)
        # Uncomment this to print out token usage each time, e.g.
        # {"completion_tokens": 86, "prompt_tokens": 26, "total_tokens": 112}
        print(completion.usage)
        return completion.choices[0].message.content

In [9]:
def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]

In [10]:
def calculate(what):
    return eval(what)

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate
}

In [11]:
query("Are there movie and tv show streaming applications that allow you to watch with multiple people at the same time?")

CompletionUsage(completion_tokens=170, prompt_tokens=317, total_tokens=487)
Chain of Thought 1:
Thought: I could search for movie and TV show streaming applications that support watching with multiple people simultaneously.
Action: wikipedia: Watch Party Streaming Apps
Pause
Observation: There are several streaming applications that offer the capability to watch movies and TV shows with multiple people at the same time, such as Netflix Party and Amazon Prime Video Watch Party.

Chain of Thought 2:
Thought 2: I should look up more details about Netflix Party and Amazon Prime Video Watch Party.
Pause
Observation 2: Both Netflix Party and Amazon Prime Video Watch Party allow users to synchronize playback and communicate through text chat while watching content together.

Answer: Yes, there are movie and TV show streaming applications like Netflix Party and Amazon Prime Video Watch Party that enable users to watch with multiple people at the same time and interact through text chat.


In [12]:
openai.__version__

'1.14.2'